In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import urlparse
import pandas as pd
from urllib.error import HTTPError
from urllib.error import URLError 
import pandas as pd
import requests
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def cosine_simi(website,term):

  X = website.lower()
  Y = term.lower()

  # tokenization
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)

  # sw contains the list of stopwords
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form a set containing keywords of both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0

  # cosine formula
  for i in range(len(rvector)):
      c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)
  print("similarity: ", cosine)
  return cosine

In [ ]:
#starting url
input_url = "https://www.google.com"

#Phrase
term = "Youtube is a video sharing platform owned by Google"
#enter number
num=3

#get url domain
current_url_domain = urlparse(input_url).netloc

#current domain
current_doamin = current_url_domain.split(".")

#list to store links and similarity_score
links=[]
similarity=[]

try:

  def crawler(input_url):
    new_url = set()

    ##create beautiful soup object to extract html tags
    html=urlopen(input_url)  
    bs=BeautifulSoup(html.read(),'html.parser')
    soup = BeautifulSoup(html)
    page = urlopen(input_url).read()
    soup = BeautifulSoup(page)

    ##access all anchor tags from input
    for link in bs.find_all('a'):
      current_link = link.get('href')

      if(current_link != "" and current_link!=None and "http" in current_link): #check that the link is not empty or not an relative link
        if(current_doamin[1] not in current_link and current_link not in links): #check that the link is not already visited
          links.append(current_link)
          print(current_link)
          for script in soup(["script", "style","a","href"]):
            script.decompose()                                                  #remove style and script tags
          strips = list(soup.stripped_strings)
          strips = ''.join(strips)                                              #join the contents into a consolidated string
          #print(strips)
          if(len(strips)==0):                                                   #if no content in the webiste, cos_similrity = 0
            cosine_sim = 0.0
            similarity.append(cosine_sim)
          else:
            cosine_sim = cosine_simi(strips,term)                               #pass the website content and the term for cosine similarity
            similarity.append(cosine_sim)                                 
          new_url.add(current_link)
          
    return new_url

  new_links = []
  new_links.append(input_url)
  for i in range(num):
    for count in range(len(new_links)):
      url = new_links.pop(0)
      urls = crawler(url)
      for j in urls:
        new_links.append(j)
        
except HTTPError as e:
  print ('HTTP Error',e)
except URLError as e:
  print('Server could not be found')

https://www.youtube.com/?gl=US&tab=w1
similarity:  0.0
https://www.youtube.com/about/
similarity:  0.20412414523193154
https://www.youtube.com/about/press/
similarity:  0.20412414523193154
https://www.youtube.com/about/copyright/
similarity:  0.20412414523193154
https://www.youtube.com/creators/
similarity:  0.20412414523193154
https://www.youtube.com/ads/
similarity:  0.20412414523193154
https://www.youtube.com/about/policies/
similarity:  0.20412414523193154
https://www.youtube.com/howyoutubeworks?utm_campaign=ytgen&utm_source=ythp&utm_medium=LeftNav&utm_content=txt&u=https%3A%2F%2Fwww.youtube.com%2Fhowyoutubeworks%3Futm_source%3Dythp%26utm_medium%3DLeftNav%26utm_campaign%3Dytgen
similarity:  0.20412414523193154
https://blog.youtube/creator-and-artist-stories/
similarity:  0.04376881095324085
https://blog.youtube/creator-and-artist-stories/crownandglory-5-questions-brooke-ashley-beverly-halls/
similarity:  0.04376881095324085
https://blog.youtube/creator-and-artist-stories/crownandgl

In [ ]:
#create df of the links
df = pd.DataFrame(
    {'links': links,
     'cosine_similarity': similarity
    })

In [ ]:
#sort the df based on cosine similarity score
simlistdf = df.sort_values(by=['cosine_similarity'], ascending=False)

In [ ]:
#reset the index of the df
simlistdf=simlistdf.reset_index(drop=True)

#remove rows where there is no match, cosine similarity is 0
simlistdf = simlistdf[simlistdf.cosine_similarity != 0]

In [ ]:
#View the df
simlistdf

,links,cosine_similarity
0,https://www.youtube.com/about/,0.204124
1,https://www.youtube.com/about/press/,0.204124
2,https://www.youtube.com/about/copyright/,0.204124
3,https://www.youtube.com/creators/,0.204124
4,https://www.youtube.com/ads/,0.204124
...,...,...
68,http://youtube.com/trends,0.043769
69,https://www.youtubego.com/,0.043769
70,https://www.youtube.com/kids/,0.043769
71,https://www.youtube.com/musicpremium,0.043769


In [ ]:
#Write the df to excel
simlistdf.to_excel("output.xlsx")